# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
city_weather = pd.read_csv('Output_data/cities.csv')
city_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Emerald,-23.5333,148.1667,84.34,45,40,19.57,AU,1636699212
1,1,Torbay,47.6666,-52.7314,39.27,86,90,34.52,CA,1636699212
2,2,Hobart,-42.8794,147.3294,52.11,89,75,1.99,AU,1636699213
3,3,Rikitea,-23.1203,-134.9692,74.16,71,42,6.80,PF,1636699213
4,4,Mercedes,-34.6515,-59.4307,61.86,82,5,8.68,AR,1636699214


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"]

In [15]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
fit_weather_data = city_weather.loc[(city_weather['Wind Speed']<= 10) & (city_weather['Cloudiness'] == 0) & 
                                    (city_weather['Max Temp']<=80) & (city_weather['Max Temp'] >= 70)]
fit_weather_data.dropna()
fit_weather_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
74,74,San Miguel,13.4833,-88.1833,71.02,89,0,2.98,SV,1636699244
149,149,Afonso Bezerra,-5.4983,-36.5056,74.86,85,0,6.82,BR,1636699276
252,252,Cabo San Lucas,22.8909,-109.9124,77.50,75,0,1.99,MX,1636699321
253,253,San Blas,21.5167,-105.2667,75.09,82,0,3.40,MX,1636699322
325,325,San Patricio,28.0170,-97.5169,71.82,66,0,8.48,US,1636699353


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
hotel_df = fit_weather_data.loc[:,:]
hotel_df['Hotel Name'] = ""

target_type = "hotel"
radius = 5000

# rewrite params dict

params = {
    "keyword" : "hotel",
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


for index, row in hotel_df.iterrows():
    city_name = row["City"]

    params["location"] = f"{row['Lat']}, {row['Lng']}"
    

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")
        

C:\Users\farim\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Retrieving Results for Index 74: San Miguel.
Closest hotel in San Miguel is Hotel Comfort Inn San Miguel.
------------


C:\Users\farim\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Retrieving Results for Index 149: Afonso Bezerra.
Closest hotel in Afonso Bezerra is Pousada Shammah.
------------
Retrieving Results for Index 252: Cabo San Lucas.
Closest hotel in Cabo San Lucas is Breathless Cabo San Lucas Resort and Spa.
------------
Retrieving Results for Index 253: San Blas.
Closest hotel in San Blas is Hotel Garza Canela.
------------
Retrieving Results for Index 325: San Patricio.
Closest hotel in San Patricio is Americas Best Value Inn Sinton.
------------
-------End of Search-------


In [44]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
74,74,San Miguel,13.4833,-88.1833,71.02,89,0,2.98,SV,1636699244,Hotel Comfort Inn San Miguel
149,149,Afonso Bezerra,-5.4983,-36.5056,74.86,85,0,6.82,BR,1636699276,Pousada Shammah
252,252,Cabo San Lucas,22.8909,-109.9124,77.50,75,0,1.99,MX,1636699321,Breathless Cabo San Lucas Resort and Spa
253,253,San Blas,21.5167,-105.2667,75.09,82,0,3.40,MX,1636699322,Hotel Garza Canela
325,325,San Patricio,28.0170,-97.5169,71.82,66,0,8.48,US,1636699353,Americas Best Value Inn Sinton


In [45]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [46]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))